# Нейронная сеть вручную vs через слои 
В этом ноутбуке рассмотрим, как работает автоматическое дифференцирование на примере двухслойной нейронной сети. Сравним два подхода: использование готовых слоев (Linear, ReLU) и реализацию тех же операций вручную через матричные умножения и активации. Покажем, что оба способа дают идентичные результаты — слои это просто удобная абстракция над математическими операциями.

Построим одну и ту же двухслойную сеть двумя способами, выполним прямой проход, вычислим градиенты и сравним результаты. Также визуализируем графы вычислений для обоих подходов. Это продемонстрирует, что класс Tensor автоматически отслеживает все операции и корректно вычисляет градиенты независимо от способа реализации.

В результате увидим, что высокоуровневый код со слоями и низкоуровневые матричные операции — это одно и то же с точки зрения вычислительного графа. Автоматическое дифференцирование применяет цепное правило ко всей композиции функций, освобождая нас от ручного вычисления градиентов.

## Немного о данной нейронной сети
$$
\begin{aligned}
& X_0 \in \mathbb{R}^{3 \times 4} \\
& W_1 \in \mathbb{R}^{4 \times 8} \\
& b_1 \in \mathbb{R}^8 \\
& X_1 = X_0 W_1 + b_1 \in \mathbb{R}^{3 \times 8} \\
& X_2 = \operatorname{ReLU}(X_1) \in \mathbb{R}^{3 \times 8} \\
& W_2 \in \mathbb{R}^{8 \times 1} \\
& b_2 \in \mathbb{R} \\
& X_3 = X_2 W_2 + b_2 \in \mathbb{R}^{3 \times 1} \\
& X_4 = \operatorname{ReLU}(X_3) \in \mathbb{R}^{3 \times 1} \\
& L = \sum_{i=1}^3 X_{4i} \in \mathbb{R}
\end{aligned}
$$


In [11]:
import sys
sys.path.append('..')

from engine.Tensor import Tensor
from engine.graph import draw_dot
from engine.Layers import Linear, ReLu

import numpy as np

## Начнем с ручного перемножения 


In [12]:
X0 = Tensor(np.random.random((3,4)))
W1 = Tensor(np.random.random((4,8)))
b1 = Tensor(np.random.random(8))
X1 = X0 @ W1 + b1
X2 = X1.relu()
W2 = Tensor(np.random.random((8,1)))
b2 = Tensor(np.random.random(1))
X3 = X2 @ W2 + b2
X4 = X3.relu()
L = X4.sum()


Сразу видим, как это неудобно. Каждое преобразование приходится писать вручную. Код уже занял кучу строк, а это лишь двухслойная сеть. При масштабировании на десятки или сотни слоев такой подход становится совершенно неуправляемым — придется объявлять сотни переменных для весов и смещений, следить за правильными размерностями матриц, не ошибиться в последовательности операций. Кроме того, код получается неструктурированным и его сложно читать — не сразу понятно, где заканчивается один слой и начинается следующий. А если нужно будет изменить архитектуру сети или добавить новый тип слоя, потребуется переписывать значительную часть кода.

Посчитаем градиенты и посмотрим на получившийся граф

In [13]:
L.backward()
draw_dot(L)

## Используем слои 

In [14]:
X = Tensor(np.random.random((3,4)))
relu = ReLu()
linear1 = Linear(4,8)
linear2 = Linear(8,1)
L = relu(linear2(relu(linear1(X)))).sum()


Теперь видим совсем другую картину. Всего в несколько строк мы описали ту же двухслойную нейронную сеть, но код стал читаемым и понятным. Архитектура сети очевидна с первого взгляда

In [15]:
L.backward()
draw_dot(L)

Главное, что граф вычислений получился абсолютно идентичным (по структуре, числа в матрицах генерировались случайно). Автоматическое дифференцирование работает одинаково в обоих случаях, потому что под капотом слои выполняют те же матричные операции. Класс Tensor не различает, была ли операция вызвана через X @ W + b или через linear(X) — для него это одна и та же последовательность умножений и сложений.

Это и есть сила автоматического дифференцирования: неважно, как вы организуете свой код — в виде элегантных высокоуровневых слоев или через низкоуровневые операции. Вычислительный граф строится автоматически, градиенты вычисляются правильно, а вам остается только сосредоточиться на архитектуре модели, а не на математических деталях её реализации.